In [19]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection, metrics, ensemble
import pyarrow.parquet as pq
import pickle
import itertools
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.ensemble import BalancedRandomForestClassifier as RandomForestClassifier
from scipy.stats import randint as sp_randint
from sklearn.utils import class_weight
import os

import plotly.graph_objects as go
from astropy.time import Time
from time import time
import json
from json import JSONEncoder

In [43]:
with open('group0.json','r') as group0:
    data_group = json.load(group0)

# print(data_group)

for index, event in enumerate(data_group):
    # print(index, event)
    # print(data_group[event])

    n_band = len(data_group[event])

    for band in range (1,n_band +1):
        # print('-'*20)
        # print(data_group[event][f'Filter {band}'])
        features = len(data_group[event][f'Filter {band}'])
        # print(features)

    # print('-'*50)

In [45]:
parquet_file = pq.ParquetFile('../lightcurves_ts.parquet')
# parquet_file.schema_arrow.names # obtienes nombre columnas
# parquet_file.metadata.num_rows # numero de filas
# parquet_file.metadata.num_columns # nummero de columnas 

print("numero de columnas:", parquet_file.metadata.num_columns)
print("numero de filas (datos): ", parquet_file.metadata.num_rows)
# print("nombre columnas: ", parquet_file.schema_arrow.names)

# se divide el paquete en grupos --> se ocupa el primer grupo 0
data = parquet_file.read_row_group(0, ['oid', 'fid', 'mjd', 'magpsf_corr', 'sigmapsf_corr', 'nbad', 'classALeRCE']).to_pandas()
# data.shape
# data.head(50)



numero de columnas: 113
numero de filas (datos):  13844917


In [46]:
# limpieza de datos 
df_data = data.replace([np.inf, -np.inf], np.nan)
df_data = df_data.dropna()
df_data.head()


print(df_data['classALeRCE'].values.size)
df_data.head()

13697564


,oid,fid,mjd,magpsf_corr,sigmapsf_corr,nbad,classALeRCE
0,ZTF17aaacfxu,1,58863.138021,20.996371,0.444790,0,RRL
1,ZTF17aaacfxu,1,58867.179803,19.374605,0.175861,0,RRL
2,ZTF17aaacfxu,1,58871.170336,19.630466,0.123111,0,RRL
3,ZTF17aaacfxu,1,58880.146204,19.362001,0.103466,0,RRL
4,ZTF17aaacfxu,1,58849.131389,19.530453,0.078387,0,RRL


In [72]:
df_data_unique = df_data.oid.unique()
limit = 10
i = 0


test = []

for index, name_event in enumerate(df_data_unique):
    # testEvent = []
    event = df_data.loc[df_data['oid'] == name_event].sort_values(by=['mjd'])

    # event = df_data.iat[df_data['oid'] == name_event]
    # event = df_data.iat[df_data['oid'] == name_event]
    rows_event, cols_event = event.shape

    if(rows_event == 1):
        continue
    test.append(event)
    # display(event)
    # print(len(event))

    # print('-'*20)
    i = i + 1
    if i == limit:
        break

# print(test)
print(len(test))

    

[            oid  fid           mjd  magpsf_corr  sigmapsf_corr  nbad  \
4  ZTF17aaacfxu    1  58849.131389    19.530453       0.078387     0   
0  ZTF17aaacfxu    1  58863.138021    20.996371       0.444790     0   
1  ZTF17aaacfxu    1  58867.179803    19.374605       0.175861     0   
2  ZTF17aaacfxu    1  58871.170336    19.630466       0.123111     0   
3  ZTF17aaacfxu    1  58880.146204    19.362001       0.103466     0   

  classALeRCE  
4         RRL  
0         RRL  
1         RRL  
2         RRL  
3         RRL  ,              oid  fid           mjd  magpsf_corr  sigmapsf_corr  nbad  \
10  ZTF17aaaemke    1  58372.401910    16.530643       0.014738     0   
45  ZTF17aaaemke    1  58388.324433    16.681208     100.000000     0   
29  ZTF17aaaemke    1  58390.353206    16.589474       0.008474     0   
58  ZTF17aaaemke    2  58431.186701    15.361944     100.000000     0   
46  ZTF17aaaemke    1  58431.211875    16.109080     100.000000     0   
..           ...  ...          

In [44]:
#Training first level of the RF model

rf_model_hierarchical = RandomForestClassifier(
            n_estimators=500,
            max_features='auto',
            max_depth=None,
            n_jobs=-1,
            bootstrap=True,
            class_weight='balanced_subsample',
            criterion='entropy',
            min_samples_split=2,
            min_samples_leaf=1)